In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ad16d20d888f95dcbb3d3cc72508815bdf4c987f0cb12ff5abdce1e50ab79441
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark.pandas as ps

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [ ]:
cols = '''movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western'''.split('|')
cols = [el.strip().replace(' ', '_').replace("'", '_').replace('-', '_') for el in cols]
cols

['movie_id',
 'movie_title',
 'release_date',
 'video_release_date',
 'IMDb_URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children_s',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film_Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci_Fi',
 'Thriller',
 'War',
 'Western']

In [ ]:
movies = ps.read_csv('films.csv', sep='|', header=False, names=cols)

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
movies.head(5)

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,2,GoldenEye (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),01-Jan-1995,None,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,None,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
movies.release_date = ps.to_datetime(movies.release_date)

In [ ]:
movies.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 1681 entries, 0 to 1680
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype     
---  ------              --------------  -----     
 0   movie_id            1681 non-null   int32     
 1   movie_title         1681 non-null   object    
 2   release_date        1680 non-null   datetime64
 3   video_release_date  0 non-null      object    
 4   IMDb_URL            1678 non-null   object    
 5   unknown             1681 non-null   int32     
 6   Action              1681 non-null   int32     
 7   Adventure           1681 non-null   int32     
 8   Animation           1681 non-null   int32     
 9   Children_s          1681 non-null   int32     
 10  Comedy              1681 non-null   int32     
 11  Crime               1681 non-null   int32     
 12  Documentary         1681 non-null   int32     
 13  Drama               1681 non-null   int32     
 14  Fantasy             1681 non-null   int32     
 15  F

In [ ]:
from datetime import datetime
fromtimestamp = lambda x: datetime.fromtimestamp(int(x))


In [ ]:
fromtimestamp(891717742)

datetime.datetime(1998, 4, 4, 19, 22, 22)

In [ ]:
ratings = ps.read_csv('ratings.csv', sep='\t', header=False, names=['user_id', 'movie_id', 'rating', 'timestamp'])

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,186,302,3,891717742
1,22,377,1,878887116
2,244,51,2,880606923
3,166,346,1,886397596
4,298,474,4,884182806


In [ ]:
ratings['time'] = ratings.timestamp.map(fromtimestamp)
ratings = ratings.drop('timestamp', axis=1)

In [ ]:
ratings.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 99999 entries, 0 to 99998
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   99999 non-null  int32         
 1   movie_id  99999 non-null  int32         
 2   rating    99999 non-null  int32         
 3   time      99999 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(3)

In [ ]:
ratings.head()

,user_id,movie_id,rating,time
0,186,302,3,1998-04-04 19:22:22
1,22,377,1,1997-11-07 07:18:36
2,244,51,2,1997-11-27 05:02:03
3,166,346,1,1998-02-02 05:33:16
4,298,474,4,1998-01-07 14:20:06


In [ ]:
user = ratings.groupby('user_id').agg(count = ('rating', 'count')).head().sort_values('count', ascending=False).head(1)
user

,count
user_id,
833,267


In [ ]:
user['count'].values[0]

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


267

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/indexes/base.py:636: UserWarning: We recommend using `Int64Index.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


833

In [ ]:
filt_ratings = ratings[ratings.user_id == user['count'].index.values[0]]

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/indexes/base.py:636: UserWarning: We recommend using `Int64Index.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
filt_ratings.head()

,user_id,movie_id,rating,time
55680,833,340,5,1997-11-18 01:58:13
55681,833,544,1,1997-09-24 20:37:38
55930,833,546,2,1997-09-23 17:39:14
56181,833,161,1,1997-09-25 21:55:15
56189,833,156,4,1997-09-23 18:19:35


In [ ]:
movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,2,GoldenEye (1995),1995-01-01,None,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,3,Four Rooms (1995),1995-01-01,None,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),1995-01-01,None,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,5,Copycat (1995),1995-01-01,None,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
4,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,1995-01-01,None,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
filt_ratings = filt_ratings.merge(movies, on='movie_id').drop(['movie_idm', 'video_release_date', 'IMDb_URL'], axis=1)
filt_ratings.head()

,user_id,movie_id,rating,time,movie_title,release_date,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western
0,833,340,5,1997-11-18 01:58:13,Boogie Nights (1997),1997-01-01,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,833,544,1,1997-09-24 20:37:38,Things to Do in Denver when You're Dead (1995),1996-02-02,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0
2,833,546,2,1997-09-23 17:39:14,Broken Arrow (1996),1996-02-09,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,833,161,1,1997-09-25 21:55:15,Top Gun (1986),1986-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,833,156,4,1997-09-23 18:19:35,Reservoir Dogs (1992),1992-01-01,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0


In [ ]:
ratings_grouped = ratings.groupby('movie_id').agg(rating_count=('rating', 'count'), sum_rate=('rating', 'sum')).to_pandas()
ratings_grouped.head()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,rating_count,sum_rate
movie_id,,
496,231,952
471,221,798
463,71,274
148,128,410
1342,2,5


In [ ]:
type(ratings_grouped)

In [ ]:
filt_ratings['rating_count'] = filt_ratings.movie_id.apply(lambda x: ratings_grouped.loc[x][0])
filt_ratings['sum_rate'] = filt_ratings.movie_id.apply(lambda x: ratings_grouped.loc[x][1])

In [ ]:
filt_ratings.head()

,user_id,movie_id,rating,time,movie_title,release_date,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,rating_count,sum_rate
0,833,340,5,1997-11-18 01:58:13,Boogie Nights (1997),1997-01-01,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,189,678
1,833,544,1,1997-09-24 20:37:38,Things to Do in Denver when You're Dead (1995),1996-02-02,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,71,224
2,833,546,2,1997-09-23 17:39:14,Broken Arrow (1996),1996-02-09,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,254,770
3,833,161,1,1997-09-25 21:55:15,Top Gun (1986),1986-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,220,766
4,833,156,4,1997-09-23 18:19:35,Reservoir Dogs (1992),1992-01-01,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,148,590


,user_id,movie_id,rating,time,movie_title,release_date,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,rating_count,sum_rate


In [ ]:
filt_ratings = filt_ratings.drop(['user_id', 'movie_id', 'time', 'movie_title'], axis=1)
filt_ratings.head()

,rating,release_date,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,rating_count,sum_rate
0,5,1997-01-01,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,189,678
1,1,1996-02-02,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,71,224
2,2,1996-02-09,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,254,770
3,1,1986-01-01,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,220,766
4,4,1992-01-01,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,148,590


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
filt_ratings.release_date = filt_ratings.release_date.map(lambda x: x.year)
filt_ratings.head()


,rating,release_date,unknown,Action,Adventure,Animation,Children_s,Comedy,Crime,Documentary,Drama,Fantasy,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,rating_count,sum_rate
0,5,1997.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,189,678
1,1,1996.0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,71,224
2,2,1996.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,254,770
3,1,1986.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,220,766
4,4,1992.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,148,590


In [ ]:
filt_ratings = filt_ratings.fillna(0)

In [ ]:
x = filt_ratings.drop('rating', axis=1).to_pandas()
y = filt_ratings['rating'].to_pandas()

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/usr/local/lib/python3.10/dist-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas Series is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
scaler = MinMaxScaler((0, 1)).fit(x_train)

In [ ]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
lin_reg = LinearRegression().fit(x_train, y_train)
rand_forest = RandomForestRegressor(n_estimators=100).fit(x_train, y_train)

In [ ]:
mae_lin_reg = mean_absolute_error(y_test, lin_reg.predict(x_test))
mae_random_fores = mean_absolute_error(y_test, rand_forest.predict(x_test))

In [ ]:
f'mae_lin_reg = {mae_lin_reg}'

'mae_lin_reg = 1.5737760627454076'

In [ ]:
f'mae_random_fores = {mae_random_fores}'

'mae_random_fores = 1.0027777777777778'

In [3]:
from pyspark.sql import SparkSession

In [4]:
from threading import local
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
['user_id', 'movie_id', 'rating', 'timestamp']

In [6]:
schema = StructType([
    StructField('user_id',
                IntegerType(), True),
    StructField('movie_id',
                IntegerType(), True),
    StructField('rating',
                IntegerType(), True),
    StructField('timestamp',
                IntegerType(), True)
])

In [ ]:
schema

StructType([StructField('user_id', IntegerType(), True), StructField('movie_id', IntegerType(), True), StructField('rating', IntegerType(), True), StructField('timestamp', IntegerType(), True)])

In [7]:
ratings = spark.read.csv('ratings.csv', schema = schema, sep='\t', header=False)

In [ ]:
ratings.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



In [ ]:
ratings.groupBy('movie_id').mean('rating').show(5)

+--------+------------------+
|movie_id|       avg(rating)|
+--------+------------------+
|     496| 4.121212121212121|
|     471|3.6108597285067874|
|     463| 3.859154929577465|
|     148|          3.203125|
|    1342|               2.5|
+--------+------------------+
only showing top 5 rows



In [8]:
movies = spark.read.csv('films.csv', sep='|', header=False, inferSchema=True)

In [ ]:
movies.show(5)

+---+-----------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|_c0|              _c1|        _c2| _c3|                 _c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+---+-----------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|  1| Toy Story (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|  2| GoldenEye (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  3|Four Rooms (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  4|Get Shorty (1995)|01-Jan-1995|NULL|http://us.imdb.co...|  0|  1|  0|  0|  0|   1|  

In [9]:
cols = '''movie id | movie title | release date | video release date |
              IMDb URL | unknown | Action | Adventure | Animation |
              Children's | Comedy | Crime | Documentary | Drama | Fantasy |
              Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
              Thriller | War | Western'''.split('|')
cols = [el.strip().replace(' ', '_').replace("'", '_').replace('-', '_') for el in cols]
cols

['movie_id',
 'movie_title',
 'release_date',
 'video_release_date',
 'IMDb_URL',
 'unknown',
 'Action',
 'Adventure',
 'Animation',
 'Children_s',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film_Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci_Fi',
 'Thriller',
 'War',
 'Western']

In [10]:
old_cols = movies.columns
dic_cols = dict(zip(old_cols, cols))

In [ ]:
dic_cols

{'_c0': 'movie_id',
 '_c1': 'movie_title',
 '_c2': 'release_date',
 '_c3': 'video_release_date',
 '_c4': 'IMDb_URL',
 '_c5': 'unknown',
 '_c6': 'Action',
 '_c7': 'Adventure',
 '_c8': 'Animation',
 '_c9': 'Children_s',
 '_c10': 'Comedy',
 '_c11': 'Crime',
 '_c12': 'Documentary',
 '_c13': 'Drama',
 '_c14': 'Fantasy',
 '_c15': 'Film_Noir',
 '_c16': 'Horror',
 '_c17': 'Musical',
 '_c18': 'Mystery',
 '_c19': 'Romance',
 '_c20': 'Sci_Fi',
 '_c21': 'Thriller',
 '_c22': 'War',
 '_c23': 'Western'}

In [11]:
movies = movies.withColumnsRenamed(dic_cols)
movies.show(5)


+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|      movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children_s|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+--------+-----------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1| Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2| GoldenEye (1995)| 01-Jan-1995|              N

In [ ]:
movies.dtypes

[('movie_id', 'int'),
 ('movie_title', 'string'),
 ('release_date', 'string'),
 ('video_release_date', 'string'),
 ('IMDb_URL', 'string'),
 ('unknown', 'int'),
 ('Action', 'int'),
 ('Adventure', 'int'),
 ('Animation', 'int'),
 ('Children_s', 'int'),
 ('Comedy', 'int'),
 ('Crime', 'int'),
 ('Documentary', 'int'),
 ('Drama', 'int'),
 ('Fantasy', 'int'),
 ('Film_Noir', 'int'),
 ('Horror', 'int'),
 ('Musical', 'int'),
 ('Mystery', 'int'),
 ('Romance', 'int'),
 ('Sci_Fi', 'int'),
 ('Thriller', 'int'),
 ('War', 'int'),
 ('Western', 'int')]

In [ ]:
?ratings.join

In [12]:
ratings_ = ratings.join(movies, on='movie_id', how='inner')
ratings_.show(5)


+--------+-------+------+---------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|user_id|rating|timestamp|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children_s|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+--------+-------+------+---------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     242|    196|     3|881250949|        Kolya (1996)| 24-Jan-1997|              NULL|http://us.imdb.co...|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|    

In [13]:
from pyspark.sql import functions as F

In [ ]:
ratings_[ratings_['unknown'] == 1].select(F.avg('rating')).take(1)[0][0]

3.2

In [ ]:
for col in movies.columns[5:]:
    print(f"avg_{col} = {ratings_[ratings_[col] == 1].select(F.round(F.avg('rating'), 2)).take(1)[0][0]}")

avg_unknown = 3.2
avg_Action = 3.48
avg_Adventure = 3.5
avg_Animation = 3.58
avg_Children_s = 3.35
avg_Comedy = 3.39
avg_Crime = 3.63
avg_Documentary = 3.67
avg_Drama = 3.69
avg_Fantasy = 3.22
avg_Film_Noir = 3.92
avg_Horror = 3.29
avg_Musical = 3.52
avg_Mystery = 3.64
avg_Romance = 3.62
avg_Sci_Fi = 3.56
avg_Thriller = 3.51
avg_War = 3.82
avg_Western = 3.61


In [14]:
ratings.show(5)

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
+-------+--------+------+---------+
only showing top 5 rows



In [23]:
ratings.groupBy('movie_id').agg({'rating': 'count'}).orderBy('count(rating)', ascending=False).show(5)

+--------+-------------+
|movie_id|count(rating)|
+--------+-------------+
|      50|          583|
|     258|          509|
|     100|          508|
|     181|          507|
|     294|          485|
+--------+-------------+
only showing top 5 rows



In [25]:
ratings.groupBy('movie_id').agg({'rating': 'count'}).orderBy('count(rating)').show(5)

+--------+-------------+
|movie_id|count(rating)|
+--------+-------------+
|    1460|            1|
|    1618|            1|
|    1339|            1|
|    1645|            1|
|    1650|            1|
+--------+-------------+
only showing top 5 rows



In [24]:
ratings.groupBy('movie_id').agg({'rating': 'avg'}).orderBy('avg(rating)', ascending=False).show(5)

+--------+-----------+
|movie_id|avg(rating)|
+--------+-----------+
|    1599|        5.0|
|    1467|        5.0|
|    1500|        5.0|
|    1653|        5.0|
|    1122|        5.0|
+--------+-----------+
only showing top 5 rows



In [26]:
ratings.groupBy('movie_id').agg({'rating': 'avg'}).orderBy('avg(rating)').show(5)

+--------+-----------+
|movie_id|avg(rating)|
+--------+-----------+
|    1339|        1.0|
|    1561|        1.0|
|    1618|        1.0|
|    1352|        1.0|
|     858|        1.0|
+--------+-----------+
only showing top 5 rows

